# Code

## Initialize

In [379]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as tsplot
import arch as arch

## Create database

In [494]:
#Create the Stock and Index price database 
st_date = dt.datetime(2019,1,1)
en_date = dt.datetime(2022,2,28)
stks_needed = ['JPM', 'SPY']#^GSPC
Stk_prices_list = []
flag = 0
for tc in list(stks_needed):
    if flag ==0 : 
        flag =1 
        Stk_prices_list.append(pdr.DataReader(tc, 'yahoo', start=st_date, end=en_date)['Adj Close'].index)
    Stk_prices_list.append(pdr.DataReader(tc, 'yahoo', start=st_date, end=en_date)['Adj Close'].tolist())

#Create Database of stock prices
dataframe = pd.DataFrame(list(map(list, zip(*Stk_prices_list))),columns=['Date','JPM','SPY'])
# dataframe.to_csv("Adjusted_close_Data.csv")


## Analyze Data and Build Up Risk Measures

In [492]:
Scalar = 100
# Estimate both your symbol and SPY (independently) log return volatility using GARCH (1,1) from data Jan 1, 2019 to Dec 31, 2021
dataframe["JPM_Log_returns"] = [np.log(i) for i in dataframe["JPM"]]
dataframe["JPM_Log_returns"]=dataframe["JPM_Log_returns"].pct_change()*Scalar
dataframe["SPY_Log_returns"] = [np.log(i) for i in dataframe["SPY"]]
dataframe["SPY_Log_returns"]=dataframe["SPY_Log_returns"].pct_change()*Scalar

Vol_JPM = np.std(dataframe['JPM_Log_returns'])
Vol_SPY = np.std(dataframe['SPY_Log_returns'])

#Check plots for Log returns 
# plt.plot(dataframe['JPM_Log_returns'])
# plt.title("JPM Log Returns")
# plt.xlabel("Time units")
# plt.ylabel("Log Returns")
# plt.show()
# # print(np.mean(dataframe['JPM_Log_returns']))
# plt.plot(dataframe['SPY_Log_returns'])
# plt.title("SPY Log Returns")
# plt.xlabel("Time units")
# plt.ylabel("Log Returns")
# plt.show()
# #HISTOGRAM
# plt.hist(dataframe['JPM_Log_returns'])
# plt.title("JPM Log Returns")
# plt.xlabel("Returns")
# plt.ylabel("Frequency")
# plt.show()
# # print(np.mean(dataframe['JPM_Log_returns']))
# plt.hist(dataframe['SPY_Log_returns'])
# plt.title("SPY Log Returns")
# plt.xlabel("Returns")
# plt.ylabel("Frequency")
# plt.show()
# print(np.mean(dataframe['SPY_Log_returns']))
# tsplot.plot_acf(dataframe['JPM_Log_returns'][1:])
# plt.show()
# tsplot.plot_acf(dataframe['SPY_Log_returns'][1:])
# plt.show()
# tsplot.plot_pacf(dataframe['JPM_Log_returns'][1:])
# plt.show()
# tsplot.plot_pacf(dataframe['SPY_Log_returns'][1:])
# plt.show()

#Get Train data based on date
Train_end_date = dt.datetime(2021,12,31)
Train_data = dataframe.loc[dataframe['Date']<=Train_end_date]
Test_data = dataframe.loc[dataframe['Date']>Train_end_date]
Total_len = len(dataframe) 
Test_len = Total_len -len(Train_data)


#Fit Garch(1,1) model to data
JPM_Garch_model = arch.arch_model(Train_data["JPM_Log_returns"][1:],mean="Zero",vol='GARCH',p=1,q=1).fit()
SPY_Garch_model = arch.arch_model(Train_data["SPY_Log_returns"][1:],mean="Zero",vol='GARCH',p=1,q=1).fit()


Iteration:      1,   Func. Count:      5,   Neg. LLF: 3149.429427588736
Iteration:      2,   Func. Count:     14,   Neg. LLF: 972.9377305783098
Iteration:      3,   Func. Count:     20,   Neg. LLF: 3053.7654373356795
Iteration:      4,   Func. Count:     26,   Neg. LLF: 260.9790241598004
Iteration:      5,   Func. Count:     30,   Neg. LLF: 260.97760493194795
Iteration:      6,   Func. Count:     34,   Neg. LLF: 260.97760343516813
Iteration:      7,   Func. Count:     37,   Neg. LLF: 260.977603435294
Optimization terminated successfully    (Exit mode 0)
            Current function value: 260.97760343516813
            Iterations: 7
            Function evaluations: 37
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      5,   Neg. LLF: 544.6370656989527
Iteration:      2,   Func. Count:     14,   Neg. LLF: 673.5658371925069
Iteration:      3,   Func. Count:     20,   Neg. LLF: 0.8625650307839692
Iteration:      4,   Func. Count:     25,   Neg. LLF: -296.3613973484

C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.06058. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [454]:
#Model Statistics
print(JPM_Garch_model.summary())

                       Zero Mean - GARCH Model Results                        
Dep. Variable:        JPM_Log_returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.001
Vol Model:                      GARCH   Log-Likelihood:               -260.978
Distribution:                  Normal   AIC:                           527.955
Method:            Maximum Likelihood   BIC:                           541.839
                                        No. Observations:                  756
Date:                Sun, Apr 10 2022   Df Residuals:                      756
Time:                        19:29:05   Df Model:                            0
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      6.0045e-03  2.385e-03      2.517  1.183e-02 [1.

In [455]:
print(SPY_Garch_model.summary())

                       Zero Mean - GARCH Model Results                        
Dep. Variable:        SPY_Log_returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.001
Vol Model:                      GARCH   Log-Likelihood:                297.679
Distribution:                  Normal   AIC:                          -589.358
Method:            Maximum Likelihood   BIC:                          -575.474
                                        No. Observations:                  756
Date:                Sun, Apr 10 2022   Df Residuals:                      756
Time:                        19:29:05   Df Model:                            0
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      1.8954e-03  5.993e-04      3.163  1.562e-03 [7.

In [456]:
#Forecast Garch Values Function - Not needed for the Project 
def Garch_Var(n, omega , alpha , beta, last_value, last_pct):
    vols = [last_value]*(n+1)
    returns = [last_pct]*(n+1)
    for i in range(1,n+1):
        vols[i] = (omega + (alpha * (returns[i-1]**2)) + (beta * vols[i-1]**2))**0.5
        returns[i] = vols[i] * np.random.normal(0,1)/Scalar # Assumung returns have mean 0 and volatility is coming from GARCH model
    return vols


In [491]:
#Get Volatility Estimates from the model results for period Dec 31st 2021 to Feb 28 2022
Jpm_return_estimates = Garch_Var(Test_len, JPM_Garch_model.params[0],JPM_Garch_model.params[1],JPM_Garch_model.params[2],np.std(Train_data["JPM_Log_returns"][1:]), Test_data["JPM_Log_returns"].reset_index(drop=True)[0])
# plt.plot([i for i in Jpm_return_estimates])# in Percentages
# plt.plot(Test_data["JPM_Log_returns"].reset_index(drop=True))
# plt.show()
# print(Jpm_return_estimates[1])
print(JPM_Garch_model.forecast(horizon=1).variance[-1:]**0.5)

Spy_return_estimates = Garch_Var(Test_len, SPY_Garch_model.params[0],SPY_Garch_model.params[1],SPY_Garch_model.params[2],np.std(Train_data["SPY_Log_returns"][1:]), Test_data["SPY_Log_returns"].reset_index(drop=True)[0]/100)
# plt.plot([i*100 for i in Spy_return_estimates])
# plt.plot(Test_data["SPY_Log_returns"].reset_index(drop=True))
# plt.show()
# print(Spy_return_estimates[1])
print(SPY_Garch_model.forecast(horizon=1).variance[-1:]**0.5)


          h.1
756  0.205546
          h.1
756  0.123926


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [461]:
#Correlation for Actual Data
Corr_actual = np.corrcoef(Train_data['JPM_Log_returns'][1:], Train_data['SPY_Log_returns'][1:])
print("Correlation estimate for actual returns (TRAIN Data): ",Corr_actual[0,1])

#Correlation for Estimated Data
Corr_Estimated = np.corrcoef(Jpm_return_estimates, Spy_return_estimates)
print("Correlation estimate for estmated returns: ",Corr_Estimated[0,1])


#Correlation for test Data
Corr_test = np.corrcoef(Test_data['JPM_Log_returns'], Test_data['SPY_Log_returns'])
print("Correlation estimate for test returns (TEST DATA): ",Corr_test[0,1])

#Model does not perform so well

#Correlation for Full Data
Corr_test = np.corrcoef(dataframe['JPM_Log_returns'][1:], dataframe['SPY_Log_returns'][1:])
print("Correlation estimate for test returns (FULL DATA): ",Corr_test[0,1])

Correlation estimate for actual returns (TRAIN Data):  0.7765645219986756
Correlation estimate for estmated returns:  0.976520127312045
Correlation estimate for test returns (TEST DATA):  0.42715098370511295
Correlation estimate for test returns (FULL DATA):  0.7648478335346234


## Portfolio Analysis

In [387]:
#Inital Asset Value 
Inv_Value_A = 1000000
Inv_Value_B = 1000000
Inv_Value_C = 2000000

#VaR percentile
VaR_Conf = 0.99
temp_percentile_99 = (Total_len - Test_len)*(1-VaR_Conf)
perc_99 = int(temp_percentile_99)
perc_weight = temp_percentile_99 - perc_99

### VaR Estimation

In [401]:
JPM_Hist_Sim = [i*Inv_Value_A for i in dataframe["JPM"].pct_change()[1:]]
JPM_Hist_Sim_Sort = JPM_Hist_Sim[:]
JPM_Hist_Sim_Sort.sort()
JPM_VaR = perc_weight * JPM_Hist_Sim_Sort[perc_99 - 1] + (1-perc_weight) * JPM_Hist_Sim_Sort[perc_99]
JPM_ES = np.mean(JPM_Hist_Sim_Sort[0:perc_99-1])
print("one day 99% Var and ES for JPM: ",JPM_VaR, JPM_ES)

SPY_Hist_Sim = [i*Inv_Value_A for i in dataframe["SPY"].pct_change()[1:]]
SPY_Hist_Sim_Sort=SPY_Hist_Sim[:]
SPY_Hist_Sim_Sort.sort()
SPY_VaR = perc_weight * SPY_Hist_Sim_Sort[perc_99 - 1] + (1-perc_weight) * SPY_Hist_Sim_Sort[perc_99]
SPY_ES = np.mean(SPY_Hist_Sim_Sort[0:perc_99-1])
print("one day 99% Var and ES for SPY: ",SPY_VaR, SPY_ES)

Port_Hist_Sim = [(SPY_Hist_Sim[i]+JPM_Hist_Sim[i]) for i in range(len(SPY_Hist_Sim))]
Port_Hist_Sim_Sort=Port_Hist_Sim[:]
Port_Hist_Sim_Sort.sort()
Port_VaR = perc_weight * Port_Hist_Sim_Sort[perc_99 - 1] + (1-perc_weight) * Port_Hist_Sim_Sort[perc_99]
Port_ES = np.mean(Port_Hist_Sim_Sort[0:perc_99-1])
print("one day 99% Var and ES for the Portfolio: ",Port_VaR, Port_ES)

one day 99% Var and ES for JPM:  -62327.298363982336 -104580.93013463018
one day 99% Var and ES for SPY:  -44964.90286477064 -73370.9909039779
one day 99% Var and ES for the Portfolio:  -98786.35572423699 -175951.2612215544


### Back testing

In [396]:
#BackTesting 
BackTesting_Start_date = dt.datetime(2021,3,1)
Backtest_data = dataframe.loc[dataframe['Date']>= BackTesting_Start_date,"JPM"]

Backtest_data = [i*Inv_Value_A for i in  Backtest_data.pct_change()[1:]]

Exceptions = [i for i in Backtest_data if i<=JPM_VaR]
[min(Backtest_data),JPM_VaR]
print("Exceptions incurred in backtesting: ",len(Exceptions)/Total_len*100,"%")

Exceptions incurred in backtesting:  0.0 %


### Normal Approximation

In [468]:
JPM_Mean = np.mean(dataframe["JPM"].pct_change()[1:])
JPM_std = np.std(dataframe["JPM"].pct_change()[1:])
print(JPM_Mean, JPM_std)
SPY_Mean = np.mean(dataframe["SPY"].pct_change()[1:])
SPY_std = np.std(dataframe["SPY"].pct_change()[1:])
print(SPY_Mean, SPY_std)

0.000815878220583063 0.022186212231809537
0.0008597313060295603 0.013760551338635348


### RAROC

In [490]:
JPM_ret_0104 = Test_data
(dataframe.iloc[756:758])

,Date,JPM,SPY,JPM_Log_returns,SPY_Log_returns
756,2021-12-31,156.248322,473.489044,-0.016242,-0.040946
757,2022-01-03,159.553833,476.230530,0.414432,0.093720
